In [1]:
import calibrator as cal
import numpy as np

In [6]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'


calibrator = cal.VectorScaling(bias=False, num_label = 100, device=device, print_verbose= True) # cal.VectorScaling(num_label=<num_classes>, bias=True)


In [7]:
logits_and_labels_file = "/teamspace/studios/this_studio/Selective_Prediction_VQA/predictions/logits_and_labels/"

In [8]:
from collections import Counter

def max_occurence(labels):
    count = Counter(labels)
    max_num = max(count, key=count.get)
    return max_num
    
def load_data(batch_no):
    file_name = "Logits_and_labels" + batch_no + ".pt"
    data = torch.load(logits_and_labels_file + file_name)
    
    labels = [ max_occurence(answer_labels) for answer_labels in data['labels']]
    logits = torch.cat(data['logits']).numpy()
    return logits, labels

In [9]:
##test calibrator

logits = torch.cat([torch.randn(1,100)]*2).numpy()
labels = np.array([ 0, 5])
calibrator.fit( logits, labels )

Before temperature - NLL: 1.531, ECE: 0.053
Loss : -1.0
Loss : 3.9512462615966797
Optimal temperature:  [-16.501505    5.4920654   3.338091    3.4838214 -13.061449  -17.267546
   4.0500593 -14.733671    4.6738296   3.0384746   4.305448  -13.9725
 -12.474006  -11.915403   27.833706  -13.506426  -13.01353   -13.347215
   2.637464    3.0673158 -14.078781    2.8757017   8.0540285   3.761711
   3.7893932   3.1502109   9.615899    4.209221    3.6093957 -12.950162
 -12.803491    3.2404373 -13.971854  -14.135305    3.9516664 -12.9927845
   3.1339004   3.0728388 -13.070023    4.7726073 -12.701356  -12.75356
   2.513749  -14.046107  -14.07385     3.8125536   3.3505492   4.4285994
   3.8055823 -13.462475  -12.353548    3.3538709   3.458338    3.2069433
 -13.004023    5.5260167   3.6003737   1.9493771   3.2391498 -12.772103
 -13.229067   27.786562  -13.029464  -12.230836    4.186655  -14.062937
   3.7421017 -13.074028  -13.48179    -9.800969  -13.166535  -13.558137
 -13.050418    3.3645308   4.649

([6.736653e-08], [0.01923077553510666])

In [25]:
vector_scaling_model_path ="/teamspace/studios/this_studio/Selective_Prediction_PathVQA/calibration_methods/vector_scaling/vector_calibrator.pt"
vector_model_dict = {
    "biasFlag" : calibrator.biasFlag,
    "temperature" : calibrator.temperature,
    "bias" : calibrator.bias,
    "weights" : calibrator.weights,
    "num_label": calibrator.num_label
}
# self, num_label, bias=False, weights=None, device=None, print_verbose=False
torch.save(vector_model_dict, vector_scaling_model_path)

In [26]:
dict = torch.load(vector_scaling_model_path)
cal2 = cal.VectorScaling(bias=dict['biasFlag'], weights= dict['weights'], num_label = dict['num_label'], device=device, print_verbose= True)
cal2.temperature = dict['temperature']
cal2.bias = dict['bias']

In [27]:
xlogits = torch.cat([torch.randn(1,100)]*2).numpy()
cal2.calibrate(xlogits)


array([[-1.3259434e-07, -1.2746599e+02,  2.1007523e+01,  6.3591427e+01,
        -2.4180031e-06, -5.1163809e-08, -3.3123482e+01,  4.8215242e-07,
        -8.1545044e+01, -3.5732384e+01, -1.1501936e+02,  2.9613491e-07,
        -2.1912688e-06, -1.6304176e-06, -1.8145190e+12,  2.8024815e-06,
         2.4338594e-06, -1.8596612e-06, -5.0862517e+00, -3.1148178e+01,
        -9.8960061e-07, -4.2183933e+01,  1.3432275e+03,  8.9826981e+01,
         9.2832270e+00, -1.5861221e+01,  3.8023000e+03, -4.6261745e+01,
        -3.1768658e+01, -6.1558279e-07,  9.8374517e-07, -2.9980921e+01,
        -1.2256232e-06, -7.4347616e-07,  5.3290775e+01,  2.1259266e-06,
        -4.5646977e+00, -4.4218502e+01,  2.2296952e-06,  1.2365869e+02,
        -4.5842548e-06,  4.4166592e-07, -3.3294635e+00,  7.8465109e-07,
         2.7495668e-07, -4.1584476e+01,  3.9503307e+00, -5.1324226e+01,
        -4.8245949e+01,  7.2299122e-07,  1.7246224e-06, -1.2573494e+01,
         3.0791300e+01,  3.1243340e+01,  8.3582910e-07, -1.25619

In [28]:
calibrator.calibrate(xlogits)

array([[-1.3259434e-07, -1.2746599e+02,  2.1007523e+01,  6.3591427e+01,
        -2.4180031e-06, -5.1163809e-08, -3.3123482e+01,  4.8215242e-07,
        -8.1545044e+01, -3.5732384e+01, -1.1501936e+02,  2.9613491e-07,
        -2.1912688e-06, -1.6304176e-06, -1.8145190e+12,  2.8024815e-06,
         2.4338594e-06, -1.8596612e-06, -5.0862517e+00, -3.1148178e+01,
        -9.8960061e-07, -4.2183933e+01,  1.3432275e+03,  8.9826981e+01,
         9.2832270e+00, -1.5861221e+01,  3.8023000e+03, -4.6261745e+01,
        -3.1768658e+01, -6.1558279e-07,  9.8374517e-07, -2.9980921e+01,
        -1.2256232e-06, -7.4347616e-07,  5.3290775e+01,  2.1259266e-06,
        -4.5646977e+00, -4.4218502e+01,  2.2296952e-06,  1.2365869e+02,
        -4.5842548e-06,  4.4166592e-07, -3.3294635e+00,  7.8465109e-07,
         2.7495668e-07, -4.1584476e+01,  3.9503307e+00, -5.1324226e+01,
        -4.8245949e+01,  7.2299122e-07,  1.7246224e-06, -1.2573494e+01,
         3.0791300e+01,  3.1243340e+01,  8.3582910e-07, -1.25619

In [16]:
print(cal2.weights)

None


In [17]:
print(calibrator.weights)

None


In [20]:
calibrator.temperature

Parameter containing:
tensor([-16.5015,   5.4921,   3.3381,   3.4838, -13.0614, -17.2675,   4.0501,
        -14.7337,   4.6738,   3.0385,   4.3054, -13.9725, -12.4740, -11.9154,
         27.8337, -13.5064, -13.0135, -13.3472,   2.6375,   3.0673, -14.0788,
          2.8757,   8.0540,   3.7617,   3.7894,   3.1502,   9.6159,   4.2092,
          3.6094, -12.9502, -12.8035,   3.2404, -13.9719, -14.1353,   3.9517,
        -12.9928,   3.1339,   3.0728, -13.0700,   4.7726, -12.7014, -12.7536,
          2.5137, -14.0461, -14.0738,   3.8126,   3.3505,   4.4286,   3.8056,
        -13.4625, -12.3535,   3.3539,   3.4583,   3.2069, -13.0040,   5.5260,
          3.6004,   1.9494,   3.2391, -12.7721, -13.2291,  27.7866, -13.0295,
        -12.2308,   4.1867, -14.0629,   3.7421, -13.0740, -13.4818,  -9.8010,
        -13.1665, -13.5581, -13.0504,   3.3645,   4.6490, -13.2796,   3.7961,
        -13.0557,   2.7697,   3.2022, -11.0062, -13.0021, -12.4510,   4.5590,
          2.9974, -12.6033, -13.1618, -14.

In [21]:
calibrator.bias

Parameter containing:
tensor([ 0.7801, -0.0130,  0.5852, -0.2616, -0.8093, -0.5417, -0.8897, -0.1742,
        -0.1931,  0.0618,  0.2524, -0.5098, -0.5404,  0.1121,  0.2397,  0.8082,
         0.9259,  0.0407,  0.6877,  0.4868, -0.4914,  0.0651,  0.0707,  0.4710,
         0.9068,  0.6635, -0.1971, -0.1383, -0.2433, -0.1021,  0.5614,  0.8891,
         0.2837,  0.3093,  0.2453,  0.9796,  0.2976, -0.8754,  0.2943,  0.4940,
        -0.9652,  0.4555,  0.1603,  0.1595, -0.6786,  0.8173,  0.3035,  0.2915,
        -0.4743, -0.1330, -0.1558,  0.7592,  0.6146, -0.5531, -0.7185, -0.6137,
        -0.1182,  0.7454,  0.6059, -0.5910,  0.5927, -0.9336, -0.6378,  0.2470,
        -0.2615,  0.4371, -0.8241, -0.3880,  0.7655, -0.7158, -0.4607,  0.5999,
        -0.9464, -0.1182,  0.3471,  0.0742, -0.8219,  0.6306,  0.6914, -0.1855,
         0.4447, -0.3772,  0.3557,  0.6744, -0.1333,  0.6606, -0.6545, -0.2148,
         0.6143, -0.7537,  0.4834, -0.8162, -0.7339, -0.0327, -0.7336,  0.8761,
         0.2939,  

In [22]:
calibrator.biasFlag

False